In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the API key
API_KEY = os.getenv("API_KEY")


My API Key is: YCJVIX0BHBQWD92T


In [4]:
import requests as r

# Base URL
url = "https://alphavantageapi.co/timeseries/analytics"

# Parameters
params = {
    "SYMBOLS": "AAPL,NVDA,IBM, DELL, GOOGL",
    "RANGE": "2024-01-01,2024-11-01",
    "INTERVAL": "DAILY",
    "OHLC": "close",
    "CALCULATIONS": "CORRELATION",
    "apikey": "API_KEY" 
}

# Make the request
response = r.get(url, params=params)

# Check the response
print(response.url)  # Verify the constructed URL
print(response.status_code)

https://alphavantageapi.co/timeseries/analytics?SYMBOLS=AAPL%2CNVDA%2CIBM%2C+DELL%2C+GOOGL&RANGE=2024-01-01%2C2024-11-01&INTERVAL=DAILY&OHLC=close&CALCULATIONS=CORRELATION&apikey=API_KEY
200


In [5]:
# extract and store in a dataframe
import pandas as pd
analytics_data = response.json()

# convert data to dataframe
corr_data = analytics_data['payload']['RETURNS_CALCULATIONS']['CORRELATION']
corr_df = pd.DataFrame(
    corr_data['correlation'], 
    index=corr_data['index'], 
    columns=corr_data['index']
)

corr_df.head()


,GOOGL,NVDA,AAPL,IBM,DELL
GOOGL,1.000000,NaN,NaN,NaN,NaN
NVDA,0.328185,1.000000,NaN,NaN,NaN
AAPL,0.378144,0.268774,1.000000,NaN,NaN
IBM,0.087962,0.163780,0.104065,1.000000,NaN
DELL,0.174169,0.488450,0.126794,0.104519,1.0


In [6]:
# Fill NaN values by filling in the reverse (symmetry of correlation matrix)
for i in range(len(corr_df.columns)):
    for j in range(i+1, len(corr_df.columns)):
        # If the lower triangle is populated but the upper triangle has NaN, fill it
        if pd.isna(corr_df.iloc[i, j]):
            corr_df.iloc[i, j] = corr_df.iloc[j, i]

corr_df.head()

,GOOGL,NVDA,AAPL,IBM,DELL
GOOGL,1.000000,0.328185,0.378144,0.087962,0.174169
NVDA,0.328185,1.000000,0.268774,0.163780,0.488450
AAPL,0.378144,0.268774,1.000000,0.104065,0.126794
IBM,0.087962,0.163780,0.104065,1.000000,0.104519
DELL,0.174169,0.488450,0.126794,0.104519,1.000000


In [7]:
# save as .csv
corr_df.to_csv('stocks_correlation.csv', index=True)